# Process subtitle text and extract part of speech considering pronoun resolution

In [1]:
# install spacy version 2.1.3 due to issues with the neural co ref library 
!pip install spacy==2.1.3


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 27.7 MB 1.3 MB/s 
     |████████████████████████████████| 3.2 MB 48.4 MB/s 
     |████████████████████████████████| 184 kB 48.4 MB/s 
     |████████████████████████████████| 2.1 MB 29.8 MB/s 
     |████████████████████████████████| 82 kB 464 kB/s 
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.3
    Uninstalling srsly-2.4.3:
      Successfully uninstalled srsly-2.4.3
  Attempting uninstall: preshed
    Found existing installation: preshed 3.0.6
    Uninstalling preshed-3.0.6:
      Successfully uninstalled preshed-3.0.6
  Attempting uninstall: blis
    Found existing installation: blis 0.7.7
    Uninstalling blis-0.7.7:
      Successfully uninstalled blis-0.7.7
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninstalling thinc-8.0.17:
      Successfully uninstalled thinc-8.0.17
  Attempting u

In [2]:
# install the neural co ref
!pip install neuralcoref

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 286 kB 8.2 MB/s 
     |████████████████████████████████| 132 kB 44.3 MB/s 
     |████████████████████████████████| 79 kB 9.6 MB/s 
     |████████████████████████████████| 8.9 MB 56.7 MB/s 
     |████████████████████████████████| 127 kB 68.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
# download the small english library
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.1 MB 9.3 MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.1.0-py3-none-any.whl size=11074433 sha256=8ff7555acd3261f3e0fe1575c483eccf1ba8edd6f42a5691175f240cbb92ec51
  Stored in directory: /tmp/pip-ephem-wheel-cache-2iz7c0pk/wheels/59/4f/8c/0dbaab09a776d1fa3740e9465078bfd903cc22f3985382b496
Successfully built en-core-web-sm
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.3.0
    Uninstalling en-core-web-sm-3.3.0:
      Successfully uninstalled en-core-web-sm-3.3.0
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [7]:
# imports
import spacy
import neuralcoref
import os
import glob

In [51]:
#  load up the small model
nlp = spacy.load("en_core_web_sm")
# pop the neural co ref module into the pipeline

neuralcoref.add_to_pipe(nlp,greedyness=0.45)


In [52]:

# quick test to check it works
doc = nlp("Tell Sam that he will have to leave without Arthur, as he is sick.")
print(doc._.coref_resolved)

Tell Sam that he will have to leave without Arthur, as he is sick.


In [90]:
# load the sub title contents into a single string.
big_string = ""
for filename in glob.glob('*.srt'):
  print(filename)
  with open(os.path.join(os.getcwd(), filename), 'r') as f:
    for row in f.read().split("\n"):
      if row == "" or row[0].isdigit():
        continue
      big_string = big_string + " " + row
big_string = (".\n").join([row.strip() for row in big_string.split(".")])

print(big_string[:500])

print("Constructing spacy doc")

doc = nlp(big_string)



4.srt
8.srt
5.srt
1.srt
9.srt
3.srt
6.srt
10.srt
2.srt
7.srt
Like all sparrows, they eat pretty well anything; insects, fruit and particularly seeds.
They convert that diet into their own flesh, which is the richest of all foods: meat.
So they themselves are much hunted.
A falcon is also looking for a meal And it has one.
Meat is such a rich food that a falcon need only kill once a day to sustain itself.
So there's plenty of time for sitting around on the perch.
Nice work if you can get it.
But getting it is not necessarily all that easy.
This hillside in
Constructing spacy doc


In [91]:
# process the doc to replace the pronouns with the referenced nouns

resolved_doc = doc._.coref_resolved
print(resolved_doc[:500]) # first 500 characters

Like all sparrows, they eat pretty well anything; insects, fruit and particularly seeds.
all sparrows convert that diet into all sparrows own flesh, which is the richest of all foods: meat.
So all sparrows themselves are much hunted.
A falcon is also looking for a meal And A falcon has one.
Meat is such a rich food that a falcon need only kill once a day to sustain a falcon.
So there's plenty of time for sitting around on the perch.
Nice work if you can get it.
But getting it is not necessarily 


In [92]:
f_out = open("subtitles_resolved.txt", "w")
f_out.write(resolved_doc)
f_out.close()

In [93]:
# load the list of bird names extracted from wikipedia

bird_dict = {}
birds = open("bird_list.txt").read().lower().split("\n")
for bird in birds:
  # skip single letters A, B, C etc
  if len(bird) == 1:
    continue
  bird_dict[bird] = True
  # insert the last word too e..g yellow crested parrot > parrot
  bird_dict[bird.split(" ")[-1]] = True

# check it has worked
list(bird_dict.keys())[:5]

["abbott's babbler", 'babbler', "abbott's booby", 'booby', "abbott's starling"]

## Extract the nouns adj and proper nouns highlighting bird names

In [94]:
# split the subtitles into a set of rows for processing
resolved_subtitles = open("subtitles_resolved.txt").read()

subtitle_rows = resolved_subtitles.split("\n")
print(subtitle_rows[:5])


['Like all sparrows, they eat pretty well anything; insects, fruit and particularly seeds.', 'all sparrows convert that diet into all sparrows own flesh, which is the richest of all foods: meat.', 'So all sparrows themselves are much hunted.', 'A falcon is also looking for a meal And A falcon has one.', 'Meat is such a rich food that a falcon need only kill once a day to sustain a falcon.']


In [123]:
import json
# dump the filtered resolved subtitles to a file

valid_tokens = ["NOUN", 'ADJ', 'PROPN']

lemma_tags = {"NNS", "NNPS"}

birds = {}
nouns = {}
adjs = {}
links = []
filtered_subtitle_file = open("subtitles_resolved_filtered.txt", "w")
for row in subtitle_rows:
  filtered_subtitle_file.write(row + "\n")
  row_doc = nlp(row)
  row_tokens = []
  for token in row_doc:
    # depluralise nouns and proper nouns
    singular = token.text
    if token.tag_ in lemma_tags:
      singular = token.lemma_
    if token.pos_ not in valid_tokens:
      continue
    is_bird = token.pos_ == "NOUN" and singular in bird_dict
    # print(singular + "\t" + token.lemma_ + "\t" + token.pos_ + "\t" + str(is_bird) + "\n")
    if is_bird:
      birds[singular] = birds.get(singular, 0) + 1
    if token.pos_ == "NOUN":
      nouns[singular] = nouns.get(singular, 0) + 1
    if token.pos_ == "ADJ":
      adjs[singular] = adjs.get(singular, 0) + 1
    row_tokens.append(singular)
    filtered_subtitle_file.write(singular + "\t" + token.lemma_ + "\t" + token.pos_ + "\t" + str(is_bird) + "\n")
  row_tokens.sort()
  for t1 in range(len(row_tokens)):
    for t2 in range(t1 + 1, len(row_tokens)):
      links.append([row_tokens[t1], row_tokens[t2]])
  filtered_subtitle_file.write("\n")
filtered_subtitle_file.close()

# dump the nouns, adjs, birds, and links
with open('nouns.json', 'w') as f:
  f.write(json.dumps(nouns))

with open('adjs.json', 'w') as f:
  f.write(json.dumps(adjs))

with open('birds.json', 'w') as f:
  f.write(json.dumps(birds))

links.sort()
with open('links.json', 'w') as f:
  f.write(json.dumps(links))
